In this experiment, we decode Shor's nine-qubit quantum error correcting code.
We show decoding of the Shor's nine-qubit code using Dephasing DMRG, which is our own built-in DMRG-like optimisation algorithm to solve the main component problem which in its turn is the problem of finding a computational basis state cotributing the most to a given state.

In [16]:
import numpy as np
import qecstruct as qec
from mdopt.mps.utils import marginalise, create_custom_product_state
from mdopt.contractor.contractor import mps_mpo_contract
from mdopt.optimiser.utils import (
    SWAP,
    COPY_LEFT,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)
from examples.decoding.decoding import (
    css_code_checks,
    css_code_logicals,
    css_code_logicals_sites,
    css_code_constraint_sites,
)
from examples.decoding.decoding import (
    apply_constraints,
    apply_bias_channel,
)

Let us first import the code from `qecstruct` and take a look at it.

In [17]:
code = qec.shor_code()
code

X stabilizers:
[0, 1, 2, 3, 4, 5]
[3, 4, 5, 6, 7, 8]
Z stabilizers:
[0, 1]
[1, 2]
[3, 4]
[4, 5]
[6, 7]
[7, 8]

This quantum error correcting code is defined on 9 physical qubits and has 2 logical operators. This means we will need $9*2 + 2 = 20$ sites in our MPS.

In [18]:
num_sites = 2 * len(code) + code.num_x_logicals() + code.num_z_logicals()
num_logicals = code.num_x_logicals() + code.num_z_logicals()
assert num_sites == 20
assert num_logicals == 2

Now, let us define the initial state. First of all we will check that no error implies no correction. This means starting from the all-zeros state followed by decoding will return all-zeros state for the logical operators (the final logical operator will be identity operator).

In [19]:
error = "000000000010000111"
string_state = "++" + error
error_state = create_custom_product_state(string=string_state, form="Right-canonical")

Here, we get the sites where the checks will be applied. We will need to construct MPOs using this data.

In [20]:
checks_x, checks_z = css_code_checks(code)
print("X checks:")
for check in checks_x:
    print(check)
print("Z checks:")
for check in checks_z:
    print(check)

X checks:
[2, 4, 6, 8, 10, 12]
[8, 10, 12, 14, 16, 18]
Z checks:
[3, 5]
[5, 7]
[9, 11]
[11, 13]
[15, 17]
[17, 19]


These lists mention only the sites where we will apply the XOR constraints. However, the MPOs will also consist of other tensors, such as SWAPs (wire crossings) and boundary XOR constraints. In what follows we define the list of these auxiliary tensors and the corresponding sites where they reside.

In [21]:
csscode_constraint_sites = css_code_constraint_sites(code)
print("Full X-check lists of sites:")
for string in csscode_constraint_sites[0]:
    print(string)
print("Full Z-check lists of sites:")
for string in csscode_constraint_sites[1]:
    print(string)

Full X-check lists of sites:
[[2], [4, 6, 8, 10], [3, 5, 7, 9, 11], [12]]
[[8], [10, 12, 14, 16], [9, 11, 13, 15, 17], [18]]
Full Z-check lists of sites:
[[3], [], [4], [5]]
[[5], [], [6], [7]]
[[9], [], [10], [11]]
[[11], [], [12], [13]]
[[15], [], [16], [17]]
[[17], [], [18], [19]]


Let us now take a look at the logical operators.

In [22]:
print(code.x_logicals_binary())
print(code.z_logicals_binary())

[0, 1, 2]

[0, 3, 6]



We need to again translate it to our MPO language by changing the indices since we add the logical-operator sites to the end of the MPS.

In [23]:
print(css_code_logicals(code)[0])
print(css_code_logicals(code)[1])

[2, 4, 6]
[3, 9, 15]


Now goes the same operation of adding sites where auxiliary tensors go.

In [24]:
print(css_code_logicals_sites(code)[0])
print(css_code_logicals_sites(code)[1])

[[0], [2, 4], [1, 3, 5], [6]]
[[1], [3, 9], [2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14], [15]]


In [25]:
tensors_constraints = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
tensors_logicals = [COPY_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
constraint_sites_logicals = css_code_logicals_sites(code)

Now the fun part, contracting the logical MPOs.

In [26]:
renormalise = True
error_state = apply_bias_channel(
    basis_mps=error_state, basis_string=string_state, prob_channel=0.1
)

In [27]:
error_state = apply_constraints(
    error_state,
    csscode_constraint_sites[0],
    tensors_constraints,
    renormalise=renormalise,
)
error_state = apply_constraints(
    error_state,
    csscode_constraint_sites[1],
    tensors_constraints,
    renormalise=renormalise,
)
error_state = apply_constraints(
    error_state,
    constraint_sites_logicals,
    tensors_logicals,
    renormalise=renormalise,
)

100%|██████████| 2/2 [00:00<00:00, 483.58it/s]


Marginalise over the message bits to get the logical operator MPS.

In [32]:
sites_to_marginalise = list(range(num_logicals, len(error_state)))
logical = marginalise(mps=error_state, sites_to_marginalise=sites_to_marginalise)
print(logical.dense(flatten=True))

[0.00110897 0.00278983 0.00142582 0.00358692]
